# Setup Environment

In [ ]:
! pip install epitran==1.26.0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.8 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10744 sha256=9d5442e17e65cdf34cadb6d4681337702fde69e9bea33a290ccb2bc88151e8b5
  Stored in directory: /root/.cache/pip/wheels/ec/03/6f/d2e0162d94c0d451556fa43dd4d5531457245c34a36b41ef4a
Successfully built unicodecsv


In [ ]:
! pip install g2pk==0.9.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.9/580.9 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 18.8 MB/s eta 0:00:00


In [ ]:
! pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.3 MB/s eta 0:00:00


In [ ]:
import os
import re
import csv
import pandas as pd
import json
import itertools
from tqdm import tqdm
from jiwer import cer

# mapping

In [ ]:
output_to_phonetics_map = {
    'м': 'm',
    'ʷ':' v',
    'w': 'v',
    'c': 'k',
    'ĉ': 'C',
    'č': 'C',
    '̕': "?",
    "'": '?',
    'ʔ': "?",
    'ꞌ': "?",
    '̛':  "?",
    '’': "?",
    'ʼ': "?",
    "'": '?',
    'â': 'A',
    'â': 'A',
    'ȃ': 'A',
    'ž': 'Z',
    'š': 'S',
    'W': 'v',
    'β': 'f',
    'е': 'e',
    '`': "?",
    'ɑ': 'A',
    'ɑ': 'A',
    'ʃ': 'S',
    'ð': 'z',
    'ɾ': 'r',
    'æ': 'a',
    'ɪ': 'e',
    'χ': 'x',
    'ɣ': 'q',
    'ʒ': 'Z',
    ':': '',
    'ː': '',
    'ā': 'A',
    'ː': '',
    'ä': 'A',
    'á': 'A',
    'š': 'S',
    'ū': 'u',
    'û': 'u',
    'ś': 's',
    'ī': 'i',
    'í': 'i',
    'î': 'i',
    'é': 'e',
    'ḥ': 'h',
    'ɒ': 'A',
    'ʰ': '',
    'ə': 'e',
    'R': 'r',
    'W': 'v',
    'Q': 'q',
    'T': 't',
    'Y': 'y',
    'P': 'p',
    'D': 'd',
    'F': 'f',
    'H': 'h',
    'J': 'j',
    'L': 'l',
    'X': 'x',
    'V': 'v',
    'B': 'b',
    'N': 'n',
    'M': 'm',
    'K': 'k',
    'G': 'g',
    'U': 'u',
    'O': 'o',
    'I': 'i',
    'E': 'e',
    'ŋ': 'ng',
    '.': '',
    'ɛ': 'e',
    'ʊ': 'u',
    "ˈ": '?',
    'ù': 'u',
    'θ': 's',
    '̪': '',
    'ũ': 'u',
    '_': '',
    'ç': 'C',
    'ĝ': 'q',
    'ɢ': 'q',
    'ː': '',
    'í': 'i',
    'ŝ': 'S',
    '!': '',
    'ǧ': 'q',
    'ʻ': '?',
    'è': 'e',
    '�': '',
    'ú': 'u',
    'ô': 'o',
    'ē': 'e',
    'à': 'A',
    'ă': 'A',
    'ǐ': 'i',
    'ü': 'u',
    '\u200e': '',
    'ğ': 'q',
    'ṣ': 'S',
    'â': 'A',
    'â': 'A',
    'ȃ': 'A',
    'ž': 'Z',
    'š': 'S',
    'ā': 'A',
    'ː': '',
    'ä': 'A',
    'á': 'A',
    'š': 'S',
    'ū': 'u',
    'û': 'u',
    'ś': 'S',
    'ī': 'i',
    'í': 'i',
    'î': 'i',
    'é': 'e',
}

consonants_regex = '(?=' + '|'.join(['q', 'r', 't', 'y', 'p', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm', 'Q', 'R', 'T', 'Y', 'P', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'Z', 'X', 'C', 'V', 'B', 'N', 'M' ]) + ')'
vowels_regex = '(?=' + '|'.join(['a', 'A', 'e', 'i', 'u', 'o']) + ')'


def replace_phonetic_characters(input_string, char_map=output_to_phonetics_map, from_phonetics=False):
    substituted = re.sub(r'tʃʰ', 'C', input_string)
    substituted = re.sub(r't͡ʃ', 'C', input_string)
    substituted = re.sub(r'tʃ', 'C', substituted)
    substituted = re.sub(r't͡S', 'C', substituted)
    substituted = re.sub(r'ow', 'o', substituted)
    substituted = re.sub('d͡ʒ', 'j', substituted)
    substituted = re.sub('dʒ', 'j', substituted)

    # Create a translation table using str.maketrans
    translation_table = str.maketrans(char_map)

    # Use str.translate to replace characters based on the translation table
    translated = substituted.translate(translation_table)

    return translated

# Get Evaluation Data

In [ ]:
!wget https://huggingface.co/datasets/MahtaFetrat/SentenceBench/raw/main/SentenceBench.csv

--2025-05-10 11:19:00--  https://huggingface.co/datasets/MahtaFetrat/SentenceBench/raw/main/SentenceBench.csv
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.55, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56026 (55K) [text/plain]
Saving to: ‘SentenceBench.csv’

SentenceBench.csv   100%[===================>]  54.71K  --.-KB/s    in 0.008s  

2025-05-10 11:19:00 (6.90 MB/s) - ‘SentenceBench.csv’ saved [56026/56026]



In [ ]:
sentence_bench = pd.read_csv('SentenceBench.csv')

In [ ]:
sentence_bench.head(3)

,dataset,grapheme,phoneme,homograph word,pronunciation
0,homograph,من قدر تو را می‌دانم,man qadr-e to rA mi-dAnam,قدر,qadr
1,homograph,از قضای الهی به قدر الهی پناه می‌برم,?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram,قدر,qadar
2,homograph,به دست و صورتم کرم زدم,be dast-o suratam kerem zadam,کرم,kerem


### Get ManaTTS

In [ ]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'mana-tts'][['grapheme', 'phoneme']]

# Convert to a list of tuples
mana_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

mana_evaluation_data[:3]

[('در این نوشته بنا داریم با یک ابزار ساده و مکانیکی افزایش بینایی برای افراد کم\u200cبینا ',
  'dar ?in neveSte banA dArim bA yek ?abzAr-e sAde va mekAniki-ye ?afzAyeS-e binAyi barAye ?afrAd-e kam\u200cbinA '),
 ('به نام بی\u200cوپتیک یا عدسی دورنما آشنا شویم. ',
  'be nAm-e biyoptik yA ?adasi-ye durnamA ?ASnA Savim'),
 ('دراین\u200cصورت، انجام خودارزیابی و ارائه بازخورد بر عهده خودتان است. ',
  'dar ?in surat ?anjAm-e xod?arzyAbi va ?erA?e-ye bAzxord bar ?ohde-ye xodetAn ?ast ')]

### Get CommonVoice

In [ ]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'commonvoice'][['grapheme', 'phoneme']]

# Convert to a list of tuples
commonvoice_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

commonvoice_evaluation_data[:3]

[('در اکثر شهرها، مرکزی برای خرید دوچرخه وجود دارد.',
  'dar ?aksar-e Sahr-hA, markazi barAye xarid-e  doCarxe vojud dArad.'),
 ('پس از مدرسه کودکان به سوی خانه جست و خیز کردند.',
  'pas ?az madrese kudakAn be suye xAne jast-o-xiz kardand.'),
 ('شما نگران زن و بچه این نباش.', 'SomA negarAn-e zan-o-baCCe-ye ?in nabAS.')]

### Get Homograph

In [ ]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'homograph'][['grapheme', 'phoneme', 'homograph word',	'pronunciation']]

# Convert to a list of tuples
homograph_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

homograph_evaluation_data[:3]

[('من قدر تو را می\u200cدانم', 'man qadr-e to rA mi-dAnam', 'قدر', 'qadr'),
 ('از قضای الهی به قدر الهی پناه می\u200cبرم',
  '?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram',
  'قدر',
  'qadar'),
 ('به دست و صورتم کرم زدم', 'be dast-o suratam kerem zadam', 'کرم', 'kerem')]

# Evaluate Method Outputs

## PER Evaluation

In [ ]:
def remove_non_word_chars(text):
    pattern = r'[^\w\s\?]'
    cleaned_text = re.sub(pattern, ' ', text)
    return cleaned_text

In [ ]:
def remove_white_spaces(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

In [ ]:
def get_word_only_text(text):
  word_only_text = remove_non_word_chars(text)
  extra_space_removed_text = remove_white_spaces(word_only_text)

  return extra_space_removed_text

In [ ]:
def get_texts_cer(reference, model_output):
  # Preprocess input texts to only contain word characters
  word_only_reference = get_word_only_text(reference)
  word_only_output = get_word_only_text(model_output)

  # Return +infinity for CER if any of the texts is empty
  if not word_only_reference.strip() or not word_only_output.strip():
    return float('inf')

  return cer(word_only_reference, word_only_output)

In [ ]:
def get_avg_cer_of_method(method_outputs, references):
  cers = []
  for idx, o in enumerate(method_outputs):
    cer = get_texts_cer(o.replace('-', ''), references[idx][1].replace('-', ''))
    if cer != float('inf'):
      cers.append(cer)

  return sum(cers) / len(cers)

## Homograph Evaluation

In [ ]:
def get_homograph_performance(outputs, references):
  corrects = 0
  total = 0

  for idx, (g, p, homograph, right) in enumerate(references):
    if homograph != '':
      total += 1
      if right in outputs[idx]:
        corrects += 1

  return corrects / total

# Full bench

In [ ]:
benchmark = []

for g, p in mana_evaluation_data:
  benchmark.append((g, p, '', ''))

for g, p in commonvoice_evaluation_data:
  benchmark.append((g, p, '', ''))

for g, p, w, r in homograph_evaluation_data:
  benchmark.append((g, p, w, r))

benchmark = benchmark[:400]

In [ ]:
def print_all_metrics(predictions):
  per = get_avg_cer_of_method(predictions, benchmark) * 100
  homograph = get_homograph_performance(predictions, benchmark) * 100

  print(f"PER: \t\t\t{per:.4f}")
  print(f"HOMOGRAPH: \t\t{homograph:.4f}")

# Epitran

In [ ]:
import epitran
epi = epitran.Epitran('fas-Arab')

In [ ]:
epi.transliterate(u'دلم میخواست برم ')

'dlm mjxvɒst brm '

In [ ]:
replace_phonetic_characters(epi.transliterate(u'دلم میخواست برم '))

'dlm mjxvAst brm '

# outputs

In [ ]:
from tqdm import tqdm
import time

outputs = []
start_time = time.time()

for g, p, _, _ in tqdm(benchmark):
    o = epi.transliterate(g)
    outputs.append(o)

total_time = time.time() - start_time
avg_time = total_time / len(benchmark) if len(benchmark) > 0 else 0
print(f"Total: {total_time:.2f}s | Avg: {avg_time:.4f}s/sample")

100%|██████████| 400/400 [00:00<00:00, 3625.70it/s]

Total: 0.12s | Avg: 0.0003s/sample


In [ ]:
mapped_outputs = []
for o in outputs:
  mapped = replace_phonetic_characters(o)
  mapped_outputs.append(mapped)
  mapped.replace('j', 'y')

In [ ]:
print_all_metrics(mapped_outputs)
print(f"TOTAL TIME:\t\t{total_time:.4f} (s)")
print(f"AVG TIME:\t\t{avg_time:.4f} (s)")

PER: 			45.1223
HOMOGRAPH: 		0.0000
TOTAL TIME:		0.1184 (s)
AVG TIME:		0.0003 (s)


# Runs

## First:

```
PER: 			45.1223
HOMOGRAPH: 		0.0000
TOTAL TIME:		0.1172 (s)
AVG TIME:		0.0003 (s)
```

## Second

```
PER: 			45.1223
HOMOGRAPH: 		0.0000
TOTAL TIME:		0.1074 (s)
AVG TIME:		0.0003 (s)
```

## Third

```
PER: 			45.1223
HOMOGRAPH: 		0.0000
TOTAL TIME:		0.1296 (s)
AVG TIME:		0.0003 (s)
```

## Fourth

```
PER: 			45.1223
HOMOGRAPH: 		0.0000
TOTAL TIME:		0.1085 (s)
AVG TIME:		0.0003 (s)
```

## Fifth

```
PER: 			45.1223
HOMOGRAPH: 		0.0000
TOTAL TIME:		0.1184 (s)
AVG TIME:		0.0003 (s)
```